Hunter Bennett | Glass Lab | Kupffer Strains Project | 14 March 2023

In addition to calling differential peaks we want to examine the mutational burden in enhancers at different log2 fold change cutoffs - idea being the mutation rate should be highest within the most strain specific set of enhancers

### Load packages

In [1]:
### header ###
__author__ = "Hunter Bennett"
__license__ = "BSD"
__email__ = "hunter.r.bennett@gmail.com"
%load_ext autoreload
%autoreload 2
### imports ###
%matplotlib inline
import os
import re
import glob
import pandas as pd
import numpy as np

Set working directory

In [2]:
workingDirectory = '/home/h1bennet/strains_github/results/Figure2_ATAC/' # user will need to set own wd
if not os.path.isdir(workingDirectory):
    os.mkdir(workingDirectory)
os.chdir(workingDirectory)

Define custom functions

In [3]:
def import_homer_diffpeak(depath, appath):
    '''This function takes in a getDiffExpression file created from raw tag counts
       and a peak tag file created with annotatePeaks
       and processes the files so that they contain the same set of genes and can be
       used for data visualization and analysis
       Accepts:
           depath (str): path to getDiffExpression file
           appath (str): path to annotatePeaks file
       Returns:
           de (pandas.DataFrame): modified getDiffExpression file
           ge (pandas.DataFrame): modified annotatePeaks,
           ge_mat (pandas.DataFrame): annotatePeaks file without annotation
           ge_mat_quatn (pandas.DataFrame) : annotatePeaks file without annotation
           and quantile normalized
       '''
    # import packages
    import pandas as pd
    import numpy as np

    # set autosomes for filtering
    autosomes = ['chr1', 'chr2', 'chr3', 'chr4',
                 'chr5', 'chr6', 'chr7', 'chr8',
                 'chr9', 'chr10', 'chr11', 'chr12',
                 'chr13', 'chr14', 'chr15', 'chr16',
                 'chr17', 'chr18', 'chr19']

    # import differential gene expression
    de = pd.read_csv(depath, sep='\t', index_col=0)
    de.index.rename('PeakID', inplace=True)
    de = de.loc[de.Chr.str.contains('|'.join(autosomes)), :]


    # import ge file
    ap = pd.read_csv(appath, sep='\t', index_col=0)
    ap.index.rename('PeakID', inplace=True)

    # select ge transcripts that are in diff_gene
    print('annotatePeaks all peaks', ap.shape)
    print('getDiffExpression selected transcripts', de.shape)
    ap = ap.loc[de.index.tolist(), :]
    print('annotatePeaks selected peaks', ap.shape)


    # return files
    return (de, ap, ap.iloc[:, 18:]) # also return count matrix without annotation

def pull_comparisons_get_diff(diff_gene, seq_type='Repeat'):
    '''This function pulls out comparisons from a diff gene file with multiple comparision groups
    and returns a dict of pandas DataFrames with one comparison each.

    Accepts:
        diff_gene (pandas.DataFrame): diff gene file processed to have
        genes as index and column of RefSeqIDs titled RepeatID
        seq_type (str): Repeat|Peak type of annotation file. repeat for RNA
        peak for ChIP/ATAC

    Returns:
    comp_dict (dict): dictionary of 1 pandas Data Frame per each comparison
    ''' 


    # import packages
    import pandas as pd
    import re
    
    if seq_type=='Repeat':
        # extract groups
        def subset_get_diff(diff_gene, comp):
            return diff_gene.loc[:, [seq_type+'ID',
                                     comp + ' Log2 Fold Change',
                                     comp + ' p-value',
                                     comp + ' adj. p-value']]

    if seq_type=='Peak':
        # exract groups
        def subset_get_diff(diff_gene, comp):
             return diff_gene.loc[:, ['Chr', 'Start', 'End',
                                      'Annotation',
                                      'Gene Name',
                                      'Distance to TSS',
                                       comp + ' Log2 Fold Change',
                                       comp + ' p-value',
                                       comp + ' adj. p-value']]
    
    comp_dict = {}
    pattern='(\w* vs. \w*).*'
    for col in diff_gene.columns.values:
        m = re.search(string=col, pattern=pattern)
        if m:
            df = subset_get_diff(diff_gene, m.group(1))
            if seq_type=='Repeat':
                df.columns = ['RepeatID', 'log2fc', 'pval', 'adj_pval']
            if seq_type=='Peak':
                df['location'] = df.Chr.astype(str)+':'+df.Start.astype(str)+'-'+df.End.astype(str)
                df.columns = ['Chr', 'Start', 'End',
                              'Annotation', 'gene', 'TSS_dist',
                              'log2fc', 'pval', 'adj_pval', 'location']
    
            comp_dict[re.sub('G0[0-9]_', '', m.group(1))] = df
    
    return comp_dict

### Import differential peak analysis

Best replicates determined by quality control in qc and annotation notebooks

In [4]:
diff_peak, peaks, peak_mat = import_homer_diffpeak(
    './diff_output.txt',
    './idr_peaks_atac_norm.txt')

# create dictionary of sub data-frames for each comparison
comp_dict = pull_comparisons_get_diff(diff_peak, seq_type='Peak')

annotatePeaks all peaks (84264, 30)
getDiffExpression selected transcripts (84264, 39)
annotatePeaks selected peaks (84264, 30)


Make output directory

In [5]:
if not os.path.isdir('./marge_mutational_burden/'):
    os.mkdir('./marge_mutational_burden/')

Set thresholds for analysis

In [6]:
pval = 0.05
fcs = [1,2,4]
peak_list_dict = {}

Generate peak lists for analysis with MMARGE

In [7]:
for fc in fcs:   
    for key in comp_dict.keys():
        # select get diff data frame.
        df = comp_dict[key]
        deg = df.index[(df.loc[:, 'adj_pval'] < pval) & (np.abs(df.loc[:, 'log2fc']) >= fc)]
        peak_list_dict[key+'_de_peaks_fc_'+str(fc)] = deg
        
        # add list for nonsig peaks
        nondeg = df.index[(df.loc[:, 'adj_pval'] >= pval) & (np.abs(df.loc[:, 'log2fc']) < fc)]
        peak_list_dict[key+'_nonsig'] = nondeg

if np.NaN get introduced into the matrix then it converts 'int' type columns to 'float' type columns, this is not ideal and interferes with downstream peak analysis so we create a dict to change the start and end columns back to integers just in case

In [8]:
convert_dict = {'Start': int,
                'End': int}

for key in peak_list_dict.keys():
    # save cell specific promoters
    tmp = diff_peak.reindex(peak_list_dict[key]).dropna(how='all').iloc[:, :4]
    tmp = tmp.astype(convert_dict)
    tmp.to_csv('./marge_mutational_burden/'+key.replace(' vs. ','_vs_')+'.txt',
               sep='\t')

Annotate mutations with MMARGE

In [9]:
with open('./marge_annotate_mutations.sh', 'w') as f:
    for peakfile in glob.glob('./marge_mutational_burden/aj*c57*.txt'):
        anno_mut = ['MMARGE.pl annotate_mutations', '-file', peakfile,
                    '-output', peakfile.replace('.txt', '_anno_muts.txt'),
                    '-ind', 'aj\n\n']
        f.write(' '.join(anno_mut))
        
    for peakfile in glob.glob('./marge_mutational_burden/balbcj*c57*.txt'):
        anno_mut = ['MMARGE.pl annotate_mutations', '-file', peakfile,
                    '-output', peakfile.replace('.txt', '_anno_muts.txt'),
                    '-ind', 'balbcj\n\n']
        f.write(' '.join(anno_mut))
        
    for peakfile in glob.glob('./marge_mutational_burden/aj*balbcj*.txt'):
        anno_mut = ['MMARGE.pl annotate_mutations', '-file', peakfile,
                    '-output', peakfile.replace('.txt', '_anno_muts.txt'),
                    '-ind', 'balbcj\n\n']
        f.write(' '.join(anno_mut))

Run the following in the command line

    bash ./marge_annotate_mutations.sh

### Calculate number of mutations in each file
First look at generated files

In [13]:
np.sort(glob.glob('./marge_mutational_burden/*anno_muts.txt'))

array(['./marge_mutational_burden/aj_vs_balbcj_de_peaks_fc_1_anno_muts.txt',
       './marge_mutational_burden/aj_vs_balbcj_de_peaks_fc_2_anno_muts.txt',
       './marge_mutational_burden/aj_vs_balbcj_de_peaks_fc_4_anno_muts.txt',
       './marge_mutational_burden/aj_vs_balbcj_nonsig_anno_muts.txt',
       './marge_mutational_burden/aj_vs_c57bl6j_de_peaks_fc_1_anno_muts.txt',
       './marge_mutational_burden/aj_vs_c57bl6j_de_peaks_fc_2_anno_muts.txt',
       './marge_mutational_burden/aj_vs_c57bl6j_de_peaks_fc_4_anno_muts.txt',
       './marge_mutational_burden/aj_vs_c57bl6j_nonsig_anno_muts.txt',
       './marge_mutational_burden/balbcj_vs_c57bl6j_de_peaks_fc_1_anno_muts.txt',
       './marge_mutational_burden/balbcj_vs_c57bl6j_de_peaks_fc_2_anno_muts.txt',
       './marge_mutational_burden/balbcj_vs_c57bl6j_de_peaks_fc_4_anno_muts.txt',
       './marge_mutational_burden/balbcj_vs_c57bl6j_nonsig_anno_muts.txt'],
      dtype='<U71')

Read in total peaks with mutation logged (non-na peaks), and peaks without a mutation logged (na peaks)

In [14]:
mut_frac_dict = {}
for peakfile in np.sort(glob.glob('./marge_mutational_burden/*anno_muts.txt')):
    mutfile = pd.read_csv(peakfile, index_col=0, sep='\t')
     #print(mutfile.iloc[:, -1].isna().value_counts())
    nomut = mutfile.iloc[:, -1].isna().sum()
    total_peaks = mutfile.iloc[:, -1].shape[0]
    mut_frac_dict[peakfile.split('/')[-1]] = round((total_peaks-nomut)/(total_peaks),
                                                   ndigits=3)

In [15]:
pd.Series(mut_frac_dict)

aj_vs_balbcj_de_peaks_fc_1_anno_muts.txt         0.263
aj_vs_balbcj_de_peaks_fc_2_anno_muts.txt         0.307
aj_vs_balbcj_de_peaks_fc_4_anno_muts.txt         0.356
aj_vs_balbcj_nonsig_anno_muts.txt                0.176
aj_vs_c57bl6j_de_peaks_fc_1_anno_muts.txt        0.552
aj_vs_c57bl6j_de_peaks_fc_2_anno_muts.txt        0.641
aj_vs_c57bl6j_de_peaks_fc_4_anno_muts.txt        0.654
aj_vs_c57bl6j_nonsig_anno_muts.txt               0.166
balbcj_vs_c57bl6j_de_peaks_fc_1_anno_muts.txt    0.491
balbcj_vs_c57bl6j_de_peaks_fc_2_anno_muts.txt    0.593
balbcj_vs_c57bl6j_de_peaks_fc_4_anno_muts.txt    0.641
balbcj_vs_c57bl6j_nonsig_anno_muts.txt           0.148
dtype: float64